In [ ]:
# !pip install qalsadi
# !pip install transformers
# !pip3 install pickle5
# !pip install pyarabic

In [1]:
import transformers
from transformers import AutoModelForTokenClassification
from transformers import AutoTokenizer
from transformers import pipeline
import torch
device = torch.device("cuda")
arabert='checkpoint-161000'
model = AutoModelForTokenClassification.from_pretrained(arabert, num_labels=5).to(device)
tokenizer = AutoTokenizer.from_pretrained(arabert)
error_classification_arabert = pipeline(task="token-classification", model=model, tokenizer=tokenizer,device=0)

C:\Users\tatas\anaconda3\envs\new_env\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import re
from pyarabic import araby
nltk.download('stopwords')
sw = stopwords.words('arabic')
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'«»'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
special_words=sw+list(punctuations_list)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tatas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
labels_map={
  'correct' : 0,
'spelling-error':1,
'grammer-error':2,
 'context-error':3,
 'seg-error':4
}

In [4]:
import pickle
import qalsadi.lemmatizer
lemmer = qalsadi.lemmatizer.Lemmatizer()
import pandas as pd
with open('all_vocab_clean (1).pickle', 'rb') as fh:
    lemma_vocab= pickle.load(fh)
omar_vocab=list(pd.read_csv('omar_vocab (1).csv')['word'])

In [5]:
df= pd.read_csv('omar_vocab (1).csv')
df=df[df['freq'] > 15]
omar_vocab=list(df['word'])

In [6]:
df

,word,freq
6,ويبستر,33
7,يضيعه,21
10,أخوتي,16
11,ستنتهي,446
14,أطيافها,43
...,...,...
477792,يمكنهم,2426
477794,استوقفت,18
477795,شقيقتها,350
477796,بالمدرجات,19


In [7]:
len(omar_vocab)

168703

In [8]:
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ه", "ة", text)
    return text

def preprocess_txt(txt):
    #deal with punct
    txt=''.join(['' if char in punctuations_list  else char for char in txt])
    #add space before and after english and numbers
    pattern=r'([0-9]+|[\u0660-\u0669]+|[a-zA-Z]+)'
    txt= re.sub(pattern,r' \1 ',txt)
    txt =araby.strip_tashkeel(txt)
    #remove multiple spaces
    txt = re.sub(' +', ' ', txt).strip()
    return txt

In [9]:
def is_number_or_en_letters(token):
    pattern=r'([0-9]+|[\u0660-\u0669]+|[a-zA-Z]+)'
    if re.match(pattern,token):
        return True
    return False

In [10]:
def get_sent_tokens_labels_scores(text,do_vocab_check=True):
    text= preprocess_txt(text)
    results=error_classification_arabert(text)
    labels=[]
    scores=[]
    words=[]
    prev_subword=''
    label=''
    subword=False
    for idx, result_dict in enumerate(results):

        if '##' not in result_dict['word']:
            if subword:
                words.append(prev_subword)
                subword=False
            # if complete word last word or next word does not contain ##
            if idx ==(len(results)-1) or '##' not in results[idx+1]['word']:
                words.append(result_dict['word'])
            else :
                prev_subword=result_dict['word']

            labels.append(int(result_dict['entity'].split('_')[1]))
            scores.append(float(result_dict['score']))


        else:
            subword = True
            prev_subword+=result_dict['word'].replace('#','')
            if idx == len(results) -1 :
                words.append(prev_subword)

    if do_vocab_check:

        for idx , word in enumerate(words):
            if is_number_or_en_letters(word):
                labels[idx] = 0
                continue

            # if label in 0 1 4 do the vocab check
            if labels[idx]  == 2:
                continue
            # context and normalized word is in omar vocab
            if labels[idx] == 3 and (normalize(word) in omar_vocab):
                labels[idx]=0
                continue

            if (word in special_words) or  (word in omar_vocab):
                labels[idx]=0
                continue

            #normalize the word and search again
            normalized_word=normalize(word)
            if  (normalized_word in special_words) or  (normalized_word in omar_vocab):
                labels[idx]=0
                continue

            #get the lemma and search in the lemmatized vocab
            token_lemma=lemmer.lemmatize(word)
            if token_lemma  in lemma_vocab:
                labels[idx] = 0
                continue
            #if not found in any vocab and the label is zero then assign it to be misspelt
            if labels[idx] == 0:
                  labels[idx]=1

    return words,labels , scores

In [11]:
get_sent_tokens_labels_scores("الجامعه الامريكيه")

(['الجامعه', 'الامريكيه'], [0, 0], [0.999437153339386, 0.999747097492218])

In [ ]:
from tables import index
import pandas as pd
import numpy as np
df=pd.read_csv(r'C:\Users\tatas\Ahram Error Detection\ahram\ahram\ahram_author\ahram_batch1.csv')

df.head()

In [ ]:
from tqdm import tqdm
wrong_words_lst=[]
wrong_words_ratio=[]
labels_lst=[]
ct = 0
i = 0



for title in tqdm(df["author"]):
#     try:
    words , labels , scores= get_sent_tokens_labels_scores(str(title),do_vocab_check=True)
    wrong_words = sum([1 if label != 0 else 0 for label in labels ])
    if len(labels) <1 :
        wrong_words=0
    wrong_words_lst.append(wrong_words)
    wrong_words_ratio.append(wrong_words /len(labels)) if len(labels) >0 else wrong_words_ratio.append(0)
    labels_lst.append(','.join(str(label) for label in labels))

    if i % 5000 == 0 and i > 0:
        new_df2 = pd.DataFrame()
        new_df2['wrong_words_count'] = wrong_words_lst
        new_df2['wrong_words_ratio'] = wrong_words_ratio
        new_df2['labels'] =labels_lst
        new_df2.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\authors\Authors ouutput Batch 1.csv',encoding='utf-8-sig',index=False)


    i += 1
#     except:
#         ct+=1
#         print(title)
#         wrong_words_lst.append('nan')
#         wrong_words_ratio.append('nan')
#         labels_lst.append('nan')
#         print(ct)

In [ ]:
new_df = df
new_df

In [ ]:
new_df['wrong_words_count'] = wrong_words_lst
new_df['wrong_words_ratio'] = wrong_words_ratio
new_df['labels'] =labels_lst

In [ ]:
new_df.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\authors\Authors ouutput Batch 1.csv',encoding='utf-8-sig',index=False)

In [ ]:
from tables import index
import pandas as pd
import numpy as np
df=pd.read_csv(r'C:\Users\tatas\Ahram Error Detection\ahram\ahram\ahram_persons\persons batch 3.csv')

df.head()

In [ ]:
df

In [ ]:
from tqdm import tqdm
wrong_words_lst=[]
wrong_words_ratio=[]
labels_lst=[]
ct = 0
i = 0



for title in tqdm(df["persons"]):
#     try:
    words , labels , scores= get_sent_tokens_labels_scores(str(title),do_vocab_check=True)
    wrong_words = sum([1 if label != 0 else 0 for label in labels ])
    if len(labels) <1 :
        wrong_words=0
    wrong_words_lst.append(wrong_words)
    wrong_words_ratio.append(wrong_words /len(labels)) if len(labels) >0 else wrong_words_ratio.append(0)
    labels_lst.append(','.join(str(label) for label in labels))

    if i % 5000 == 0 and i > 0:
        new_df2 = pd.DataFrame()
        new_df2['wrong_words_count'] = wrong_words_lst
        new_df2['wrong_words_ratio'] = wrong_words_ratio
        new_df2['labels'] =labels_lst
        new_df2.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\Persons\Persons ouutput Batch 3.csv',encoding='utf-8-sig',index=False)


    i += 1
#     except:
#         ct+=1
#         print(title)
#         wrong_words_lst.append('nan')
#         wrong_words_ratio.append('nan')
#         labels_lst.append('nan')
#         print(ct)

In [ ]:
new_df = df
new_df

In [ ]:
wrong_words_lst

In [ ]:
new_df['wrong_words_count'] = wrong_words_lst
new_df['wrong_words_ratio'] = wrong_words_ratio
new_df['labels'] =labels_lst

In [ ]:
new_df

In [ ]:
new_df.head()

In [ ]:
new_df.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\Persons\Persons ouutput Batch 3.csv',encoding='utf-8-sig',index=False)

In [ ]:
df=pd.read_csv(r'C:\Users\tatas\Ahram Error Detection\ahram\ahram\ahram_keywords\Keywords batch 3.csv')

In [ ]:
from tqdm import tqdm
wrong_words_lst=[]
wrong_words_ratio=[]
labels_lst=[]
ct = 0
i = 0



for title in tqdm(df["keywords"]):
#     try:
    words , labels , scores= get_sent_tokens_labels_scores(str(title),do_vocab_check=True)
    wrong_words = sum([1 if label != 0 else 0 for label in labels ])
    if len(labels) <1 :
        wrong_words=0
    wrong_words_lst.append(wrong_words)
    wrong_words_ratio.append(wrong_words /len(labels)) if len(labels) >0 else wrong_words_ratio.append(0)
    labels_lst.append(','.join(str(label) for label in labels))

    if i % 5000 == 0 and i > 0:
        new_df2 = pd.DataFrame()
        new_df2['wrong_words_count'] = wrong_words_lst
        new_df2['wrong_words_ratio'] = wrong_words_ratio
        new_df2['labels'] =labels_lst
        new_df2.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\keywords\keywords ouutput Batch 3.csv',encoding='utf-8-sig',index=False)


    i += 1
#     except:
#         ct+=1
#         print(title)
#         wrong_words_lst.append('nan')
#         wrong_words_ratio.append('nan')
#         labels_lst.append('nan')
#         print(ct)

In [ ]:
new_df = df
new_df

In [ ]:
new_df['wrong_words_count'] = wrong_words_lst
new_df['wrong_words_ratio'] = wrong_words_ratio
new_df['labels'] =labels_lst

In [ ]:
new_df.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\keywords\keywords ouutput Batch 3.csv',encoding='utf-8-sig',index=False)

In [12]:
df=pd.read_csv(r'C:\Users\tatas\Ahram Error Detection\ahram\ahram\ahram_places\places batch 6.csv')

In [13]:
from tqdm import tqdm
wrong_words_lst=[]
wrong_words_ratio=[]
labels_lst=[]
ct = 0
i = 0



for title in tqdm(df["places"]):
#     try:
    words , labels , scores= get_sent_tokens_labels_scores(str(title),do_vocab_check=True)
    wrong_words = sum([1 if label != 0 else 0 for label in labels ])
    if len(labels) <1 :
        wrong_words=0
    wrong_words_lst.append(wrong_words)
    wrong_words_ratio.append(wrong_words /len(labels)) if len(labels) >0 else wrong_words_ratio.append(0)
    labels_lst.append(','.join(str(label) for label in labels))

    if i % 5000 == 0 and i > 0:
        new_df2 = pd.DataFrame()
        new_df2['wrong_words_count'] = wrong_words_lst
        new_df2['wrong_words_ratio'] = wrong_words_ratio
        new_df2['labels'] =labels_lst
        new_df2.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\places\places ouutput Batch 6.csv',encoding='utf-8-sig',index=False)


    i += 1 
#     except:
#         ct+=1
#         print(title)
#         wrong_words_lst.append('nan')
#         wrong_words_ratio.append('nan')
#         labels_lst.append('nan')
#         print(ct)

  0%|                                                                            | 8/774189 [00:00<10:58:22, 19.60it/s]C:\Users\tatas\anaconda3\envs\new_env\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████| 774189/774189 [4:53:35<00:00, 43.95it/s]


In [14]:
new_df = df
new_df

,DocKey,keywords,persons,places
0,ahram_1980_01_04_page_0013_art5,"نظام الضرائب,ادم سميث,التكافل الاجتماعي,تجاره ...","المسلمين,ادم سميث,سامي رمضان",NaN
1,ahram_1980_01_04_page_0013_art6,"عبد الرحمن,خطبه الجمعه اليوم,جنوب شرق اسيا,الا...","المسلمين,مسلمي,الامين العام,موتمر,مايسه عبد ال...",ماليزيا
2,ahram_1980_01_04_page_0013_art6,"عبد الرحمن,خطبه الجمعه اليوم,جنوب شرق اسيا,الا...","المسلمين,مسلمي,الامين العام,موتمر,مايسه عبد ال...",منطقه جنوب شرق اسيا
3,ahram_1980_01_04_page_0013_art6,"عبد الرحمن,خطبه الجمعه اليوم,جنوب شرق اسيا,الا...","المسلمين,مسلمي,الامين العام,موتمر,مايسه عبد ال...",منطقه جنوب شرقي اسيا
4,ahram_1980_01_04_page_0013_art7,"مصر الجديده,ميدان الاسماعيليه",NaN,NaN
...,...,...,...,...
774184,ahram_2020_10_23_page_0019_art1,"قداسه البابا,دير مارمينا,القداس الالهي,دكتور م...","الانبا,البابا,نيافه,الا",بلاد
774185,ahram_2020_10_23_page_0019_art1,"قداسه البابا,دير مارمينا,القداس الالهي,دكتور م...","الانبا,البابا,نيافه,الا",دير مارمينا
774186,ahram_2020_10_23_page_0019_art36,"نقابه المهندسين بالغربيه,مقر النقابه,اداره الم...",NaN,الامين
774187,ahram_2020_10_23_page_0019_art36,"نقابه المهندسين بالغربيه,مقر النقابه,اداره الم...",NaN,النقابه


In [15]:
new_df['wrong_words_count'] = wrong_words_lst
new_df['wrong_words_ratio'] = wrong_words_ratio
new_df['labels'] =labels_lst

In [16]:
new_df.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\places\places ouutput Batch 6.csv',encoding='utf-8-sig',index=False)

In [ ]:
we

In [ ]:
df=pd.read_csv(r'C:\Users\tatas\Ahram Error Detection\ahram\ahram\ahram_author\ahram_batch2.csv')

In [ ]:
df

In [ ]:
from tqdm import tqdm
wrong_words_lst=[]
wrong_words_ratio=[]
labels_lst=[]
ct = 0
i = 0



for title in tqdm(df["author"]):
#     try:
    words , labels , scores= get_sent_tokens_labels_scores(str(title),do_vocab_check=True)
    wrong_words = sum([1 if label != 0 else 0 for label in labels ])
    if len(labels) <1 :
        wrong_words=0
    wrong_words_lst.append(wrong_words)
    wrong_words_ratio.append(wrong_words /len(labels)) if len(labels) >0 else wrong_words_ratio.append(0)
    labels_lst.append(','.join(str(label) for label in labels))

    if i % 5000 == 0 and i > 0:
        new_df2 = pd.DataFrame()
        new_df2['wrong_words_count'] = wrong_words_lst
        new_df2['wrong_words_ratio'] = wrong_words_ratio
        new_df2['labels'] =labels_lst
        new_df2.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\authors\authors ouutput Batch 2.csv',encoding='utf-8-sig',index=False)


    i += 1
#     except:
#         ct+=1
#         print(title)
#         wrong_words_lst.append('nan')
#         wrong_words_ratio.append('nan')
#         labels_lst.append('nan')
#         print(ct)

In [ ]:
new_df = df

In [ ]:
new_df['wrong_words_count'] = wrong_words_lst
new_df['wrong_words_ratio'] = wrong_words_ratio
new_df['labels'] =labels_lst

In [ ]:
new_df.to_csv(r'C:\Users\tatas\Ahram Error Detection\Output\authors\authors ouutput Batch 2.csv',encoding='utf-8-sig',index=False)

In [ ]:
Orion,Opus,Amapa,CMTL,Massar,RCMA,Merritt,Caravela Energy,Quantix,Third Street Ag,Valent,Volt,Verus,Ancora,Haidar Jupiter,Hondius,NWone,Ospraie,Quantix,Arion,GZC,Pinnacle Six One,Sarkis,Skylar,Viribus,Drakewood,Alphidence